In [22]:
import tensorflow as tf
import numpy as np 
import csv
import datetime
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [10]:
dataset_path = 'model/keypoint_classifier/keypoint.csv'
model_save_path = 'model/keypoint_classifier/keypoint_classifier.hdf5'
NUM_CLASSES = 4

In [11]:
X_dataset = np.loadtxt(dataset_path, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))
y_dataset = np.loadtxt(dataset_path, delimiter=',', dtype='int32', usecols=(0))
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=RANDOM_SEED)

In [17]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * 2, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

In [18]:
model.summary()  # tf.keras.utils.plot_model(model, show_shapes=True)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout_2 (Dropout)         (None, 42)                0         
                                                                 
 dense_3 (Dense)             (None, 20)                860       
                                                                 
 dropout_3 (Dropout)         (None, 20)                0         
                                                                 
 dense_4 (Dense)             (None, 10)                210       
                                                                 
 dense_5 (Dense)             (None, 4)                 44        
                                                                 
Total params: 1,114
Trainable params: 1,114
Non-trainable params: 0
_________________________________________________________________


In [19]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [20]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [23]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
%tensorboard --logdir logs/fit

In [26]:
model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=64,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback, tensorboard_callback]
)

Epoch 1/10


2022-10-15 18:32:03.331218: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


 1/57 [..............................] - ETA: 14s - loss: 1.7214 - accuracy: 0.0938
Epoch 1: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
57/57 [==============================] - 0s 3ms/step - loss: 1.4610 - accuracy: 0.2354 - val_loss: 1.2641 - val_accuracy: 0.4119
Epoch 2/10
 1/57 [..............................] - ETA: 0s - loss: 1.2890 - accuracy: 0.4062
Epoch 2: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
57/57 [==============================] - 0s 1ms/step - loss: 1.2898 - accuracy: 0.3864 - val_loss: 1.1600 - val_accuracy: 0.5806
Epoch 3/10
 1/57 [..............................] - ETA: 0s - loss: 1.2200 - accuracy: 0.4062
Epoch 3: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
57/57 [==============================] - 0s 1ms/step - loss: 1.1831 - accuracy: 0.4766 - val_loss: 1.0344 - val_accuracy: 0.6742
Epoch 4/10
 1/57 [..............................] - ETA: 0s - loss: 1.1135 - accuracy: 0.5156
Epoch 4: saving mod

In [ ]:
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)

10/10 [==============================] - 0s 984us/step - loss: 0.5694 - accuracy: 0.8521


In [ ]:
model = tf.keras.models.load_model(model_save_path)

In [ ]:
predict_result = model.predict(np.array([X_test[0]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))
print(y_test[0])

1/1 [==============================] - 0s 21ms/step
[0.7998339  0.12271202 0.02541777 0.05203631]
0
0
